# Open EFT file
We open the file using `NanoEventsFactory` from the `coffea` package.

In [1]:
import os
import numpy as np
import awkward as ak

#import matplotlib
#import matplotlib.pyplot as plt
#import mplhep as hep

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

#fontsize=12
#plt.style.use([hep.style.ROOT, {'font.size': fontsize}])
#plt.rcParams.update({'font.size': fontsize})

In [2]:
fname = "/work/mmarcheg/AnalysisConfigs/configs/ttHbb/semileptonic/eft/test_eft_1.root"
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    #metadata=metadata,
).events()

/work/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_genPartIdx => GenPart
  warnings.warn(
/work/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_jetIdx => Jet
  warnings.warn(
/work/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_photonIdx => Photon
  warnings.warn(
/work/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  warnings.warn(
/work/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index 

## Print file branches

In [3]:
events.fields

['GenMET',
 'Generator',
 'GenIsolatedPhoton',
 'LHEWeight',
 'PSWeight',
 'run',
 'GenJet',
 'luminosityBlock',
 'GenVisTau',
 'GenPart',
 'genWeight',
 'LHEReweightingWeight',
 'MET',
 'event',
 'LHE',
 'GenDressedLepton',
 'LHEPart',
 'GenVtx',
 'GenJetAK8',
 'LHEPdfWeight',
 'HTXS',
 'LHEScaleWeight']

## Print EFT reweighting weights
The EFT per-event weights are stored in the branch `LHEReweightingWeight`.

In [4]:
events.LHEReweightingWeight

<Array [[1.07, 1.13, 1, ... 1, 0.997, 0.997]] type='5000 * [var * float32[parame...'>

In [5]:
# Print the first weight, for all events
events.LHEReweightingWeight[:,0]

<Array [1.07, 1.07, 1.07, ... 1.07, 1.07, 1.07] type='5000 * float32[parameters=...'>

In [6]:
# Print the first weight, for the first event
events.LHEReweightingWeight[0,0]

1.06610107421875

## Important variables for EFT studies

The important branches are the generator level variables such as `GenJet`, `GenMET`, `GenPart`.

In [7]:
events.GenJet.pt

<Array [[132, 99.2, 81.9, ... 17.3, 14.5]] type='5000 * var * float32[parameters...'>

In [8]:
events.GenPart

<GenParticleArray [[GenParticle, ... GenParticle]] type='5000 * var * genParticle'>

In [9]:
events.GenMET

<MissingETArray [MissingET, MissingET, ... MissingET] type='5000 * missingET'>